<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/0126_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
docs = ["오늘 동물원에서 원숭이를 봤어", 
       "오늘 동물원에서 코끼리를 봤어 봤어",
       "동물원에서 원숭이에게 바나나를 줬어 바나나를"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(docs)
tfidf

<3x8 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [ ]:
type(tfidf)

scipy.sparse.csr.csr_matrix

In [ ]:
tfidf.todense()

matrix([[0.37311881, 0.        , 0.4804584 , 0.4804584 , 0.63174505,
         0.        , 0.        , 0.        ],
        [0.28680065, 0.        , 0.73861611, 0.36930805, 0.        ,
         0.        , 0.        , 0.48559571],
        [0.2344005 , 0.79374908, 0.        , 0.        , 0.        ,
         0.39687454, 0.39687454, 0.        ]])

In [ ]:
tfidf_vect.get_feature_names_out ()

array(['동물원에서', '바나나를', '봤어', '오늘', '원숭이를', '원숭이에게', '줬어', '코끼리를'],
      dtype=object)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
BoW = count_vect.fit_transform(docs)
BoW.todense()
tfidf_trans = TfidfTransformer()
tfidf = tfidf_trans.fit_transform(BoW)
tfidf.todense()

matrix([[0.37311881, 0.        , 0.4804584 , 0.4804584 , 0.63174505,
         0.        , 0.        , 0.        ],
        [0.28680065, 0.        , 0.73861611, 0.36930805, 0.        ,
         0.        , 0.        , 0.48559571],
        [0.2344005 , 0.79374908, 0.        , 0.        , 0.        ,
         0.39687454, 0.39687454, 0.        ]])

In [ ]:
import pandas as pd

vocab = count_vect.get_feature_names_out()
pd.DataFrame(tfidf.todense(), columns=vocab)

,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,0.373119,0.000000,0.480458,0.480458,0.631745,0.000000,0.000000,0.000000
1,0.286801,0.000000,0.738616,0.369308,0.000000,0.000000,0.000000,0.485596
2,0.234400,0.793749,0.000000,0.000000,0.000000,0.396875,0.396875,0.000000


In [ ]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel

doc_ls = [doc.split() for doc in docs]
id2word = corpora.Dictionary(doc_ls)
DTM = [id2word.doc2bow(doc) for doc in doc_ls]
model = TfidfModel(DTM)
tfidf = model[DTM]
tfidf[0]

[(1, 0.32718457421365993), (2, 0.32718457421365993), (3, 0.8865102981879297)]

In [ ]:
for key in id2word.keys():
    print("{} : {}".format(key, id2word[key]))

0 : 동물원에서
1 : 봤어
2 : 오늘
3 : 원숭이를
4 : 코끼리를
5 : 바나나를
6 : 원숭이에게
7 : 줬어


In [ ]:
type(tfidf)

gensim.interfaces.TransformedCorpus

In [ ]:
from gensim.matutils import sparse2full

vocab = [id2word[key] for key in id2word.keys()]
DTM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(DTM_matrix, columns=vocab)

,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.327185,0.327185,0.88651,0.000000,0.000000,0.000000,0.000000
1,0.0,0.569307,0.284654,0.00000,0.771272,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.00000,0.000000,0.816497,0.408248,0.408248


In [ ]:
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()


def get_news(url):
    headers = {"user-agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article = html.select_one("#articleBodyContents").text

                  
    article = " ".join([token[0] for token in mecab.pos(article) if token[1].strip() in ["NNG", "NNP", "VV"]])
    
    return article

urls = ["https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728"]

docs = []
for url in urls:
    docs.append(get_news(url))


In [ ]:
for doc in docs:
    print(doc[:30])

과기 정통부 유영민 장관 참석 기념행사 투입 여종 데이
한국전력공사 한국전력공사 제공 나주 연합뉴스 송 기자 
판교 자율 주행 실증 자율 주행 데이터 개방 사업 선정
통계청 텔레콤 모바일 빅 데이터 분석 들 회복세 코로나
금융 결제 원 신용 정보원 지정 비씨 카드 추가 포함 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vect.fit_transform(docs)

In [ ]:
count_vect.vocabulary_

{'과기': 48,
 '정통부': 406,
 '유영민': 328,
 '장관': 375,
 '참석': 453,
 '기념행사': 77,
 '투입': 494,
 '여종': 292,
 '구축': 61,
 '민간': 162,
 '클라우드': 479,
 '외부': 315,
 '연계': 298,
 '체계': 460,
 '개방': 16,
 '강화': 14,
 '데일리': 124,
 '이재운': 353,
 '국가': 63,
 '차원': 449,
 '시대': 263,
 '산업': 213,
 '창출': 455,
 '기존': 85,
 '변화': 178,
 '이르': 344,
 '혁신': 539,
 '분야': 192,
 '경제': 33,
 '발전': 167,
 '정부': 404,
 '청사진': 457,
 '현실': 540,
 '구현': 62,
 '계획': 38,
 '과학': 49,
 '기술': 80,
 '정보': 402,
 '통신부': 491,
 '서울': 230,
 '중구': 428,
 '대한': 120,
 '상공': 214,
 '회의소': 559,
 '생태': 225,
 '조성': 416,
 '성장': 237,
 '기반': 79,
 '마련': 139,
 '플랫': 512,
 '출범식': 471,
 '행사': 530,
 '개최': 21,
 '노웅래': 99,
 '국회': 67,
 '방송': 170,
 '통신': 490,
 '위원회': 326,
 '위원장': 325,
 '참가': 451,
 '이미지': 346,
 '픽사': 515,
 '베이': 174,
 '이루': 343,
 '문재': 154,
 '핵심': 529,
 '요소': 317,
 '문재인': 155,
 '대통령': 119,
 '보호': 182,
 '보안': 180,
 '중요': 432,
 '강조': 13,
 '맥락': 148,
 '공공': 40,
 '협업': 544,
 '양질': 284,
 '생산': 224,
 '수집': 254,
 '분석': 191,
 '유통': 330,
 '역할': 295,
 '담당': 110,
 '기관

In [ ]:
tfidf_tranformer = TfidfTransformer()
tfidf_tranformer.fit(word_count_vector)

TfidfTransformer()

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.data, keywords.col), reverse=True)

def extract_keywords(feature_name, sorted_keywords, n=5):
    return [(feature_name[idx], score) for score, idx in sorted_keywords[:n] ]

In [ ]:
feature_name = count_vect.get_feature_names_out()
tf_idf_vector = tfidf_tranformer.transform(count_vect.transform([docs[0]]))
sorted_keywords = sort_keywords(tf_idf_vector.tocoo())
keywords = extract_keywords(feature_name, sorted_keywords)
keywords



[('플랫', 0.2630245621515846),
 ('계획', 0.22544962470135824),
 ('정통부', 0.18787468725113188),
 ('과기', 0.18787468725113188),
 ('혁신', 0.15157611471898802)]

In [ ]:
for i in range(len(docs)):
    tf_idf_vector = tfidf_tranformer.transform(count_vect.transform([docs[i]]))
    sorted_keywords = sort_keywords(tf_idf_vector.tocoo())
    keywords = extract_keywords(feature_name, sorted_keywords)
    print(f"문서 {i}"
    print(keywords)


문서 1
[('플랫', 0.2630245621515846), ('계획', 0.22544962470135824), ('정통부', 0.18787468725113188), ('과기', 0.18787468725113188), ('혁신', 0.15157611471898802), ('분야', 0.15157611471898802), ('경제', 0.15157611471898802), ('구축', 0.15098624117699064), ('여종', 0.1502997498009055), ('생산', 0.1502997498009055)]

문서 2
[('한전', 0.3835353685325492), ('전력', 0.3835353685325492), ('대회', 0.2301212211195295), ('서비스', 0.17286170921899544), ('제공', 0.15411486374360572), ('기업', 0.15411486374360572), ('개발', 0.15411486374360572), ('한국전력공사', 0.15341414741301967), ('학생', 0.15341414741301967), ('일반인', 0.15341414741301967)]

문서 3
[('주행', 0.6610125681327058), ('자율', 0.6334703777938431), ('경기도', 0.22220865745662136), ('실증', 0.13771095169431372), ('개방', 0.11110432872831068), ('판교', 0.08262657101658823), ('지원', 0.0666625972369864), ('발전', 0.0666625972369864), ('뉴스', 0.0666625972369864), ('포털', 0.05508438067772549)]

문서 4
[('인구', 0.4009591863854239), ('발생', 0.29860739716747037), ('이동', 0.2775871290360627), ('지역', 0.273723447403